#  Chapter 2

### Getting started

In [39]:
import torch

In [40]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [41]:
x.shape # dimensions of each axis

torch.Size([12])

In [42]:
x.numel() # product of shape (total elements)

12

In [43]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [44]:
# using -1 to fill in dimension
# x.reshape(6,-1)
x.reshape(-1,3)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [45]:
torch.randn(3, 4) # random initialisation

tensor([[-0.2079, -0.4458, -0.8139,  0.5100],
        [-1.0873, -0.4757, -1.1398,  0.5334],
        [-1.3178, -0.6014, -0.5484,  1.6071]])

In [46]:
torch.tensor([[[1]],[[7]]]).shape

torch.Size([2, 1, 1])

In [47]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y, torch.exp(x)  # The ** operator is exponentiation

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]),
 tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03]))

In [48]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)
# similar to numpy!

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [49]:
X == Y # familiar!

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

### Broadcasting

In [51]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a +  b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [61]:
a = torch.arange(12).reshape((2, 6))
b = torch.arange(2).reshape((2, 1)) 
a + b

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 7,  8,  9, 10, 11, 12]])

### Indexing

In [64]:
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [65]:
X[-1], X[1:3,1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 5.,  6.],
         [ 9., 10.]]))

In [66]:
X[1, 2] = 999
X

tensor([[  0.,   1.,   2.,   3.],
        [  4.,   5., 999.,   7.],
        [  8.,   9.,  10.,  11.]])

In [67]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

### Saving memory

In [68]:
# after the operation Y points to a different memory location
before = id(Y)
Y = Y + X
id(Y) == before

False

In [72]:
# using [:] means the same memory location !
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 4940822800
id(Z): 4940822800


### to numpy and back

In [75]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)